In [ ]:
# Clear GPU memory
import gc
import torch

# Delete any existing models
try:
    del model
    del base_model
    del trainer
except:
    pass

# Clear GPU cache
gc.collect()
torch.cuda.empty_cache()

print("Memory cleared!")

In [ ]:
!pip install transformers accelerate bitsandbytes datasets peft trl

In [ ]:
!pip install -U bitsandbytes

In [ ]:
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from trl import SFTTrainer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch

model_name = "Qwen/Qwen2.5-1.5B"

# Load dataset
train_data = load_dataset("json", data_files="train.jsonl", split="train")
val_data = load_dataset("json", data_files="val.jsonl", split="train")

# 4-bit quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)

# Prepare model for training
model = prepare_model_for_kbit_training(model)

# LoRA configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
)

# Apply LoRA
model = get_peft_model(model, peft_config)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Format function
def format_example(example):
    return f"Instruction: {example['instruction']}\nAnswer: {example['output']}"

# Training arguments
training_args = TrainingArguments(
    output_dir="./qwen2.5-1.5b-kids-ft",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    num_train_epochs=6,
    logging_steps=50,
    eval_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=200,
    bf16=True,
    report_to="none",
)

# Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    formatting_func=format_example,
    args=training_args,
)

trainer.train()

# Save model and tokenizer
trainer.save_model("./qwen2.5-1.5b-kids-ft")
tokenizer.save_pretrained("./qwen2.5-1.5b-kids-ft")

In [ ]:
# After training, view the results
import pandas as pd
import matplotlib.pyplot as plt

history = pd.DataFrame(trainer.state.log_history)

train_logs = history[history['loss'].notna()]
eval_logs = history[history['eval_loss'].notna()]

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(train_logs['step'], train_logs['loss'], marker='o')
plt.xlabel('Steps')
plt.ylabel('Training Loss')
plt.title('Training Loss')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(eval_logs['step'], eval_logs['eval_loss'], marker='o', color='orange')
plt.xlabel('Steps')
plt.ylabel('Validation Loss')
plt.title('Validation Loss')
plt.grid(True)

plt.tight_layout()
plt.show()

print(f"Final Training Loss: {train_logs['loss'].iloc[-1]:.4f}")
print(f"Final Validation Loss: {eval_logs['eval_loss'].iloc[-1]:.4f}")
print(f"Best Validation Loss: {eval_logs['eval_loss'].min():.4f}")

In [ ]:
# zip an ddownload the model
import shutil

shutil.make_archive('my_qwen_model', 'zip', './qwen2.5-1.5b-kids-ft')

from google.colab import files
files.download('my_qwen_model.zip')

In [ ]:
import os

print("Checking directory structure...\n")

# Check main output directory
print("Contents of ./qwen2.5-1.5b-kids-ft-continued/:")
if os.path.exists("./qwen2.5-1.5b-kids-ft-continued/"):
    print(os.listdir("./qwen2.5-1.5b-kids-ft-continued/"))
else:
    print("Directory not found!")

print("\n" + "="*80 + "\n")

# Check if checkpoint (example checkpoint-200) exists
print("Contents of ./qwen2.5-1.5b-kids-ft-continued/checkpoint-200/:")
if os.path.exists("./qwen2.5-1.5b-kids-ft-continued/checkpoint-200/"):
    print(os.listdir("./qwen2.5-1.5b-kids-ft-continued/checkpoint-200/"))
else:
    print("Directory not found!")

print("\n" + "="*80 + "\n")

# Check original model
print("Contents of ./qwen2.5-1.5b-kids-ft/:")
if os.path.exists("./qwen2.5-1.5b-kids-ft/"):
    print(os.listdir("./qwen2.5-1.5b-kids-ft/"))
else:
    print("Directory not found!")

In [ ]:
# OPtional: If running on colab with new session then place the model in working directory and unzip.
!unzip /content/my_qwen_model.zip

In [ ]:
import json
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

# Path to your fine-tuned model checkpoint
finetuned_model_path = r"/content/checkpoint-200"

# Path to original Qwen model
original_model_path = "Qwen/Qwen2.5-1.5B"

print("Loading models...")

# Load fine-tuned model + tokenizer
print("Loading fine-tuned model...")
finetuned_tokenizer = AutoTokenizer.from_pretrained(finetuned_model_path)
finetuned_model = AutoModelForCausalLM.from_pretrained(
    finetuned_model_path,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

# Load original Qwen model + tokenizer
print("Loading original Qwen 2.5 1.5B model...")
original_tokenizer = AutoTokenizer.from_pretrained(original_model_path)
original_model = AutoModelForCausalLM.from_pretrained(
    original_model_path,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

# Set pad_token to avoid warning messages
if finetuned_tokenizer.pad_token is None:
    finetuned_tokenizer.pad_token = finetuned_tokenizer.eos_token
if original_tokenizer.pad_token is None:
    original_tokenizer.pad_token = original_tokenizer.eos_token

print("Models loaded successfully!\n")

# Load test dataset
test_data = load_dataset("json", data_files={"test": "test.jsonl"})["test"]

def generate_answer(instruction, model, tokenizer):
    prompt = f"Instruction: {instruction}\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt", padding=True).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            temperature=0.2,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id
        )

    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract only the answer part (after "Answer:")
    if "Answer:" in full_response:
        return full_response.split("Answer:")[-1].strip()
    return full_response

print("Running evaluation...\n")

# Evaluate both models
original_correct = 0
finetuned_correct = 0
total = 0

for item in test_data:
    instruction = item["instruction"]
    expected = item["output"]

    # Generate predictions from both models
    original_prediction = generate_answer(instruction, original_model, original_tokenizer)
    finetuned_prediction = generate_answer(instruction, finetuned_model, finetuned_tokenizer)

    # Basic correctness check
    expected_keyword = expected.split()[0].lower()

    if expected_keyword in original_prediction.lower():
        original_correct += 1
    if expected_keyword in finetuned_prediction.lower():
        finetuned_correct += 1

    total += 1

original_accuracy = original_correct / total * 100
finetuned_accuracy = finetuned_correct / total * 100

print(f"Original Model Accuracy (keyword match): {original_accuracy:.2f}%")
print(f"Fine-tuned Model Accuracy (keyword match): {finetuned_accuracy:.2f}%")
print(f"Improvement: {finetuned_accuracy - original_accuracy:.2f}%\n")

# Show sample predictions in table format
print("=" * 100)

for i in range(min(5, len(test_data))):
    item = test_data[i]
    instruction = item["instruction"]

    original_pred = generate_answer(instruction, original_model, original_tokenizer)
    finetuned_pred = generate_answer(instruction, finetuned_model, finetuned_tokenizer)

    print("instruction: ", instruction)
    print("actual pred: ", original_pred)
    print("finetuned_pred: ", finetuned_pred)
    print("="*100)

print("=" * 100)